In [ ]:
import os
import pandas as pd
import numpy as np

# Define the work hardening function
def work_hardening(true_stress, true_strain):
    if len(true_stress) != len(true_strain):
        raise ValueError("true_stress and true_strain must have the same length")
    if len(true_stress) < 2:
        return np.array([])
    return np.diff(true_stress) / np.diff(true_strain)

# Set the path to your main directory in Google Drive
main_directory = "/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers"

# Loop through each folder inside the main directory
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)

    # Check if it is a folder
    if os.path.isdir(folder_path):
        print("coming inside")
        excel_file_path = os.path.join(folder_path, "Eng Values.xlsx")
        excel_file_path_2 = os.path.join(folder_path,"fitted_hard_rate_cal.xlsx")
        # Check if the Eng Values.xlsx file exists
        print(excel_file_path)
        if os.path.exists(excel_file_path) and not os.path.exists(excel_file_path_2):
            try:
                # Read the Excel file
                data = pd.read_excel(excel_file_path)
                print("coming to data")
                # Ensure required columns exist
                if 't.stress' not in data.columns and 't.strain' not in data.columns:
                    data['t.stress'] = data['eng.stress'] * (1 + data['eng.strain'])
                    data['t.strain'] = np.log(1 + data['eng.strain'])

                # Filter valid values
                data = data[(data['t.stress'] > 0) & (data['t.strain'] > 0)]

                # Polynomial fitting
                degree = 5
                coefficients = np.polyfit(data['t.strain'], data['t.stress'], degree)
                polynomial = np.poly1d(coefficients)

                # Generate fitted values
                new_true_strain = np.linspace(data['t.strain'].min(), data['t.strain'].max() - 0.01, 100)
                fitted_true_stress = polynomial(new_true_strain)

                # Calculate Work Hardening Rate
                work_hardening_rate = work_hardening(fitted_true_stress, new_true_strain)

                # Prepare the DataFrame
                fitted_data = pd.DataFrame({'t.strain': new_true_strain, 't.stress': fitted_true_stress})
                fitted_data['hard.Rate'] = np.append(work_hardening_rate, np.nan)

                # Remove negative hardening rates
                fitted_data = fitted_data[fitted_data['hard.Rate'] > 0].reset_index(drop=True)

                # Save the output Excel file in the same folder
                output_path = os.path.join(folder_path, "fitted_hard_rate_cal.xlsx")
                fitted_data.to_excel(output_path, index=False)

                print(f"Processed: {folder_name}")

            except Exception as e:
                print(f"Error processing {folder_name}: {e}")

print("All folders processed successfully!")

coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-280/Eng Values.xlsx
coming to data
Processed: paper-280
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-278/Eng Values.xlsx
coming to data
Processed: paper-278
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-277/Eng Values.xlsx
coming to data
Processed: paper-277
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-281/Eng Values.xlsx
coming to data
Processed: paper-281
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-274/Eng Values.xlsx
coming to data
Error processing paper-274: 'eng.stress'
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-276/Eng Values.xlsx
coming to data
Processed: paper-276
coming inside
/content/drive/My Drive/Colab Notebooks/Material_Science/March_papers/paper-275/Eng Values

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
